In [1]:
import numpy as np
import tensorflow as tf

#importar los datos:
#importar desde 3 archivos diferentes, labels x,y x archivo.
#x_train,y_train = datostrain.x,datostrain.y
#x_validation, y_validation = datosvalidation.x, datosvalidation.y
#x_testing, y_testing = datostesting.x,datostesting.y
x_train=np.array([[1.0, 0.0, 2.0, 3.0, 0.0, 1.0, 1.0, 3.0, 4.0],[0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0]],dtype=np.float32)
batch_size=2


In [2]:
# Dimensiones arreglo de data. 
N,D = x_train.shape

# Redimensionar para usar tensores de Nx1 y Dx1
x_train = x_train.reshape(N,1,D,1)

# Input de la red, None es para utilzar cualquier numero de input
x_input = tf.placeholder(tf.float32, shape=[batch_size,1,D,1], name='x-input')

# Output de la red
target = tf.placeholder(tf.float32, name='target', shape=(batch_size, 10))

# filtro
filter_size, n_filters = 3, 12
# Son n_filters filtros de tamaño filter_size


In [3]:
# Model blocks
def conv_layer(input_tensor, kernel_shape, layer_name):
    # input_tensor b01c
    # kernel_shape 01-in-out
    weights = tf.get_variable("weights", kernel_shape,
                               initializer = tf.contrib.layers.xavier_initializer_conv2d())
    biases = tf.get_variable("biases", [kernel_shape[3]],
                             initializer=tf.constant_initializer(0.05))
    
    tf.summary.histogram(layer_name + "/weights", weights)
    tf.summary.histogram(layer_name + "/biases", biases)
    
    # Other options are to use He et. al init. for weights and 0.01 
    # to init. biases.
    conv = tf.nn.conv2d(input_tensor, weights, 
                       strides = [1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

def fc_layer(input_tensor, weights_shape, layer_name):
    # weights_shape in-out
    weights = tf.get_variable("weights", weights_shape,
                              initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable("biases", [weights_shape[1]],
                             initializer=tf.constant_initializer(0.0))
    tf.summary.histogram(layer_name + "/weights", weights)
    tf.summary.histogram(layer_name + "/biases", biases)
    mult_out = tf.matmul(input_tensor, weights)
    return tf.nn.relu(mult_out+biases)

In [4]:
layer_name='conv1'

with tf.variable_scope(layer_name):
    conv1_out = conv_layer(x_train, [1, filter_size, 1, n_filters], layer_name)
with tf.name_scope('pool1'):
    pool1_out = tf.nn.max_pool(conv1_out, ksize=[1, 2, 1, 1],
                          strides=[1, 2, 1, 1], padding='SAME',
                          name='pool1')
    print(pool1_out.shape)

(2, 1, 9, 12)


In [5]:
print(pool1_out)

Tensor("pool1/pool1:0", shape=(2, 1, 9, 12), dtype=float32)
